In [86]:
import numpy as np
import mosek
import cvxpy as cp
import matplotlib.pyplot as plt
from scipy.linalg import expm,cholesky,block_diag

Ref: On the design of terminal ingredients for data-driven MPC

Proposition 10 in the paper states how to design the terminal ingredients to stabilize the closed-loop system
$ \left[ \begin{array}{cc}\left(\tau \bar{P}_{\Delta}^{w}-\left[\begin{array}{cc}\mathcal{X} & 0 \\ 0 & 0\end{array}\right]\right)& \left[\begin{array}{c}A^{\prime} \mathcal{X}+B^{\prime} M \\ \mathcal{X} \\ M\end{array}\right] & 0 \\ \star & -\mathcal{X} & {\left[\begin{array}{c}Q_{r} \mathcal{X} \\ R_{r} M\end{array}\right]^{\top}} \\ \star & \star & -I\end{array} \right]\prec 0$ 

Then: $P:=\mathcal{X}^{-1}-T_{y}^{\top} Q T_{y}, K:=M \mathcal{X}^{-1}$

In [28]:
def disc_linear_system(A, B, delta_t):
    '''
    Discrete a linear system with implicit Euler
    x[k+1] = (I - delta_t * A)^{-1} @ x[k] + (I - delta_t * A)^{-1} @ (delta_t * B) @ u[k]
    Returns:
        Ak
        Bk
    '''
    Nx = np.shape(A)[0]
    Ix = np.identity(Nx)

    Ak = expm(A*delta_t)
    Bk = (expm(A * delta_t) - Ix) @ np.linalg.inv(A) @ B

    
    def disc_linear_fn(x, u):
        x_next = Ak @ x + Bk @ u
        
        return x_next
    
    
    return disc_linear_fn

In [29]:
# Collect data as initial condition
def data_generation(sys, nx, nu, N, state_init = None, input_seq = None):
    # N is the length of collected data
    # Modify this for measurement functions (y = Cx)
    if input_seq is None:
        input_seq = np.random.uniform(-1,1,(nu, N))
#         input_seq = np.random.rand(nu, N)  
    N_it = np.shape(input_seq)[1] # Number of iteration
    if state_init is None:
        state_init = np.random.rand(nx,1) 
    state_seq = state_init
    xk = state_init
    for _ in range(N_it):
        uk = input_seq[:,[_]]
        state_next = sys(xk, uk)
        state_seq = np.concatenate((state_seq, state_next),axis=1)
        xk = state_next
    # Remark: the length of state_seq is larger than the input_seq by one.
    return state_seq, input_seq

In [30]:
# Construct Hankel matrix
def Hankel(sequence, L):
    n, T = np.shape(sequence)
    print(n,T)
    Hankel_matrix = np.empty((n * L, 0), dtype = float)
    for _ in range(T-L+1):
        Hankel_matrix = np.concatenate((Hankel_matrix, sequence[: , _ : _+L ].reshape(-1,1, order = 'F')),axis = 1)
    return Hankel_matrix

In [290]:
input_seq

array([[-0.63744449, -0.1860328 ,  0.52130528,  0.77835202, -0.84784831,
        -0.76981865, -0.62378987, -0.56382597, -0.93386286,  0.25351951,
        -0.2223579 , -0.20476486, -0.19258865, -0.32608359,  0.39663205,
         0.15952003,  0.31239566,  0.33514681,  0.02646   , -0.80101688,
         0.66915118,  0.96207741, -0.4478528 ,  0.10136053,  0.35118515,
         0.85945787, -0.18372272, -0.66684544,  0.10832873, -0.25808865,
        -0.84172885,  0.96479336, -0.12387389, -0.67730907,  0.0913358 ,
        -0.62404653, -0.18446009,  0.92069789, -0.91636063,  0.93400667,
        -0.21260741, -0.99228815, -0.74147929,  0.13859394,  0.37010851,
        -0.60683375,  0.21725669, -0.50178181,  0.35871861,  0.44197301,
        -0.97552489, -0.55872079, -0.94960398, -0.84892001, -0.07699947,
        -0.28729743,  0.06893962,  0.57799319,  0.07410081, -0.88452706,
        -0.82883314, -0.40999183, -0.86189609,  0.59536181, -0.73693367,
        -0.10134175,  0.70288897, -0.30098107, -0.0

In [322]:
def Xi_stack(nu, ny, l, N, input_seq, output_seq):
    Xi = np.empty((l * (nu + ny), 0), dtype = float)
    Xi_p = np.empty((l * (nu + ny), 0), dtype = float)
    
    for i in range(N-l):
        input_temp = np.vstack(input_seq[:,i:i+l].flatten().tolist())
        output_temp = np.vstack(output_seq[:,i:i+l].flatten().tolist())
        xi_temp = np.concatenate([input_temp, output_temp], axis = 0)
        Xi = np.concatenate([Xi,xi_temp],axis = 1)
        
        input_temp = np.vstack(input_seq[:,i+1:i+l+1].flatten().tolist())
        output_temp = np.vstack(output_seq[:,i+1:i+l+1].flatten().tolist())
        xi_p_temp = np.concatenate([input_temp, output_temp], axis = 0)
        Xi_p = np.concatenate([Xi_p,xi_p_temp],axis = 1)
        
    return Xi,Xi_p

In [ ]:
# def U_stack(nu, l, N, input_seq):
#     U = np.empty((l * (nu + ny), 0), dtype = float)

    
#     for i in range(N-l):
#         input_temp = np.vstack(input_seq[:,i:i+l].flatten().tolist())
#         output_temp = np.vstack(output_seq[:,i:i+l].flatten().tolist())
#         xi_temp = np.concatenate([input_temp, output_temp], axis = 0)
#         Xi = np.concatenate([Xi,xi_temp],axis = 1)
        
        
#     return U

In [456]:
def define_LMI(nu, ny, l, N, A_prime, B_prime, Qr, Rr):
    n_xi = (nu + ny) * l
    
    
    Xi_para = cp.Parameter((n_xi, N-l)) # N-1-ell+1 = n-ell
    Xi_p_para = cp.Parameter((n_xi, N-l)) # shifted window
    U_para = cp.Parameter((nu,N-l)) 
    Z_para = cp.vstack([Xi_para,U_para])
    
    Bw = np.concatenate((np.zeros([n_xi - ny, ny]), np.eye(ny)),axis = 0) 
    Bw_pseudo_inv = np.linalg.pinv(Bw)
    
    P_delta_w = cp.hstack([cp.vstack([-Z_para @ Z_para.T, Bw_pseudo_inv @ Xi_p_para @ Z_para.T]), cp.vstack([Z_para @ Xi_p_para.T @ Bw, -Bw_pseudo_inv @ Xi_p_para @ Xi_p_para.T @ Bw])])
    
    mul_r = np.concatenate([np.concatenate([np.zeros([n_xi +nu, n_xi]), Bw_pseudo_inv],axis = 0), np.concatenate([np.eye(n_xi + nu),np.zeros([ny, n_xi + nu])],axis = 0)],axis = 1)
    P_bar_delta_w = mul_r.T @ P_delta_w @ mul_r
    
    X_cal = cp.Variable((n_xi,n_xi),PSD = True)
    Gamma = cp.Variable((n_xi,n_xi),PSD = True)
    M = cp.Variable((nu,n_xi))
    tau = cp.Variable(nonneg=True)
    gamma = cp.Variable(nonneg=True) # gamma should be positive/test: set as constraint
    
    Gamma_X_cal_stack = cp.hstack([cp.vstack([Gamma, np.eye(n_xi)]),cp.vstack([np.eye(n_xi), X_cal])])
    
    X_cal_ext = cp.hstack([cp.vstack([X_cal,np.zeros([n_xi+nu,n_xi])]),cp.vstack([np.zeros([n_xi,n_xi+nu]),np.zeros([n_xi+nu,n_xi+nu])])])
    A_B_stack = cp.vstack([A_prime @ X_cal + B_prime @ M, X_cal, M])
    Qr_Rr_stack = cp.vstack([Qr @ X_cal, Rr @ M])

    LMI_stack_r1 = cp.hstack(((tau* P_bar_delta_w - X_cal_ext), A_B_stack, np.zeros([2 * n_xi + nu, n_xi + nu])))
    LMI_stack_r2 = cp.hstack([A_B_stack.T, - X_cal, Qr_Rr_stack.T])
    LMI_stack_r3 = cp.hstack([np.zeros([n_xi + nu, 2 * n_xi + nu]), Qr_Rr_stack, -np.eye(n_xi + nu)])
    LMI_stack = cp.vstack([LMI_stack_r1, LMI_stack_r2, LMI_stack_r3])
    obj = cp.Minimize(0)
    constraints = []
#     constraints += [cp.trace(Gamma) <= gamma ** 2]
    constraints += [Gamma_X_cal_stack >= 0]
    constraints += [LMI_stack <= 0]
    
    
    prob = cp.Problem(obj, constraints)
    
    return prob, Xi_para, Xi_p_para, U_para, Z_para, X_cal, M
    
    

In [457]:
def disc_four_tank(x, u):
    A = np.array([[0.921,  0, 0.041, 0], [0, 0.918, 0, 0.033], [0, 0, 0.924, 0], [0, 0, 0, 0.937]])
    B = np.array([[0.017, 0.001], [0.001, 0.023], [0, 0.061], [0.072, 0]])
    x_next = A@x + B@u

    return x_next

In [424]:
def define_A_B_prime(nu, ny, l):
    
    blocks = np.multiply.outer(np.ones((l-1)), np.eye(nu))
    offset = nu
    aux = np.empty((0, offset), int)
    
    A11 = block_diag(aux, *blocks, aux.T)
    A12 = np.zeros((l * nu, l * ny))
    A21 = np.zeros((l * ny, l * nu))
    
    blocks = np.multiply.outer(np.ones((l-1)), np.eye(ny))
    offset = ny
    aux = np.empty((0, offset), int)
    A22 = block_diag(aux, *blocks, aux.T)
    
    A_prime = np.concatenate([np.concatenate([A11,A12],axis = 1), np.concatenate([A21,A22],axis = 1)], axis = 0)
    
    B1 = np.concatenate([np.zeros([(l-1) * nu, nu]),np.eye(nu)],axis = 0)
    B2 = np.zeros([l * ny, nu])
    B_prime = np.concatenate([B1,B2], axis = 0)
    
  
    return A_prime, B_prime
#     D1 = [np.eye(nu)] * (l-1)
#     print(D1)
#     A_prime = np.diag(D1, 1)
    
#     print(A_prime)
    
    

In [ ]:
# Formulate optimization problem
def define_opt(nu, ny, Q, R, U_Hankel, Y_Hankel, N, L, l, r = None, uub = None, ulb = None, yub = None, ylb = None, P = None, beta = None):
    # Let the reference traj r be a list setpoint for input and output. To be modified, if tracking traj required.
    
    u_decision = cp.Variable(((L+l)*nu,1))
    y_decision = cp.Variable(((L+l)*ny,1))
    
    
    alpha = cp.Variable((T-(L+l)+1,1))
    
    u_init_para = cp.Parameter((l * nu, 1))
    y_init_para = cp.Parameter((l * ny, 1))
    
    
    loss_func = 0
    for _ in range(L):
        loss_func += cp.quad_form(y_decision[ ( l + _ ) * ny : ( l + _ + 1 ) * ny ,:] - np.array(r[:ny]).reshape(-1,1), Q)
        loss_func += cp.quad_form(u_decision[ ( l + _ ) * nu : ( l + _ + 1 ) * nu ,:] - np.array(r[ny:]).reshape(-1,1), R)

    obj = cp.Minimize(loss_func)
    # Define constraints
    constraint = []
    
    constraint += [ u_decision == U_Hankel @ alpha ]
    constraint += [ y_decision == Y_Hankel @ alpha ]
    constraint += [ u_decision[: n * nu] == u_init_para ]
    constraint += [ y_decision[: n * ny] == y_init_para ]
    constraint += [ u_decision[- n * nu :] == u_ns ]
    constraint += [ y_decision[- n * ny :] == y_ns ]
    
    if uub is not None:
        for _ in range(L):
            constraint += [u_decision[ ( n + _ ) * nu : ( n + _ + 1 ) * nu, : ] <= uub]
    
    if ulb is not None:
        for _ in range(L):
            constraint += [u_decision[ ( n + _ ) * nu : ( n + _ + 1 ) * nu, : ] >= ulb]

    if yub is not None:
        for _ in range(L):
            constraint += [y_decision[ ( n + _ ) * ny : ( n + _ + 1 ) * ny ,:] <= yub]
            
    if ylb is not None:
        for _ in range(L):
            constraint += [y_decision[ ( n + _ ) * ny : ( n + _ + 1 ) * ny ,:] >= ylb]
            
    prob = cp.Problem(obj,constraint)
    
    return prob, alpha, u_init_para, y_init_para, u_decision, y_decision

In [425]:
state_seq, input_seq = data_generation(disc_four_tank, nx, nu, N)

In [426]:
state_seq

array([[ 7.80581007e-01,  7.18380973e-01,  6.92494246e-01,
         6.51065026e-01,  6.30347979e-01,  6.10100636e-01,
         5.60072225e-01,  5.43598668e-01,  5.18545567e-01,
         4.99368573e-01,  4.59712606e-01,  4.42477606e-01,
         4.20858213e-01,  3.87604768e-01,  3.53817387e-01,
         3.14458546e-01,  2.74657370e-01,  2.66735787e-01,
         2.55768760e-01,  2.35582787e-01,  2.18845186e-01,
         1.86605321e-01,  1.59303252e-01,  1.58589393e-01,
         1.36607888e-01,  1.24145810e-01,  1.24625382e-01,
         1.08879162e-01,  9.41631992e-02,  7.55749458e-02,
         8.85496225e-02,  7.15137147e-02,  8.10485653e-02,
         7.74563401e-02,  7.31301127e-02,  6.69963364e-02,
         7.09868938e-02,  8.41950878e-02,  8.49879852e-02,
         7.79786120e-02,  6.32545805e-02,  6.15018272e-02,
         7.24385714e-02,  8.41917647e-02,  9.36166775e-02,
         8.31866304e-02,  8.61758147e-02,  6.59019769e-02,
         6.94521050e-02,  7.27484159e-02,  5.86915627e-0

In [427]:
A = np.array([[0.921,  0, 0.041, 0], [0, 0.918, 0, 0.033], [0, 0, 0.924, 0], [0, 0, 0, 0.937]])
B = np.array([[0.017, 0.001], [0.001, 0.023], [0, 0.061], [0.072, 0]])
C = np.array([[1,0,0,0],[0,1,0,0]])




# disc_sys_fn = disc_linear_fn
nx = 4
nu = 2
ny = 2

Q = np.diag([1] * ny)
R = np.diag([5e-3] * nu)



l = 2
L = 30
# T = (nu + 1) * (n + L + nx) - 1
N = 100

u_ns = np.array([[1,1]* n]).reshape(-1,1)  # terminal equalibrium constraints
y_ns = np.array([[0.65,0.77]*n]).reshape(-1,1)

state_init = np.array([[0.1],[0.1],[0.2],[0.2]])

# state_seq, input_seq = data_generation(disc_four_tank, nx, nu, T, state_init = state_init)
state_seq, input_seq = data_generation(disc_four_tank, nx, nu, N)

output_seq = C @ state_seq

In [428]:
n_xi = (nu + ny) * l
n_xi

8

In [429]:
Ty = np.concatenate([np.zeros((ny, n_xi - ny)),np.eye(ny)], axis = 1)#[0 ... 0 I]
Qr = Ty.T @ Q @ Ty # since Ty.T @ Q @ Ty = diag(0,0,1,1)
L = cholesky(R)
Rr = L.T # Since L @ L.T.conj = A if chol(A)

In [430]:
A_prime, B_prime = define_A_B_prime(nu, ny, l)
A_prime

array([[0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

In [459]:
 prob, Xi_para, Xi_p_para, U_para, Z_para, X_cal, M= define_LMI(nu, ny, l, N, A_prime, B_prime, Qr, Rr)

In [460]:
Xi, Xi_p = Xi_stack(nu,ny,l,N,input_seq,output_seq)

In [461]:
U = input_seq[:,l:]

In [462]:
Z = np.concatenate([Xi, U], axis = 0)

In [463]:
Xi_para.value = Xi
Xi_p_para.value = Xi_p
U_para.value = U 
# Z_para.value = Z

In [464]:
prob.solve(solver=cp.MOSEK)

0.0

In [465]:
K = M.value @ np.linalg.inv(X_cal.value)
P = np.linalg.inv(X_cal.value) - Ty.T @ Q @ Ty

In [466]:
P

array([[ 1.10375401e+11, -3.48251603e+10, -5.72162854e+09,
        -5.72162854e+09, -3.48251604e+10, -3.48251604e+10,
        -3.14224522e+09, -3.14224522e+09],
       [-3.48251603e+10,  1.10375401e+11, -5.72162854e+09,
        -5.72162855e+09, -3.48251603e+10, -3.48251605e+10,
        -3.14224523e+09, -3.14224522e+09],
       [-5.72162854e+09, -5.72162854e+09,  2.69844002e+10,
        -5.26312694e+09, -5.72162856e+09, -5.72162856e+09,
        -1.21023372e+10, -1.21023372e+10],
       [-5.72162854e+09, -5.72162855e+09, -5.26312694e+09,
         2.69844002e+10, -5.72162856e+09, -5.72162855e+09,
        -1.21023372e+10, -1.21023372e+10],
       [-3.48251604e+10, -3.48251603e+10, -5.72162856e+09,
        -5.72162856e+09,  1.10375401e+11, -3.48251603e+10,
        -3.14224524e+09, -3.14224523e+09],
       [-3.48251604e+10, -3.48251605e+10, -5.72162856e+09,
        -5.72162855e+09, -3.48251603e+10,  1.10375401e+11,
        -3.14224522e+09, -3.14224523e+09],
       [-3.14224522e+09, -3.142245